In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math
import re
 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer,KNNImputer
 
import warnings
 
warnings.filterwarnings('ignore')
 
sns.set_style('darkgrid')

In [ ]:
data = pd.read_csv('/content/drive/My Drive/ColabDataset/Flight Fare/Concatenate_B2C_B2E.csv',low_memory=True)

In [ ]:
data.head(10)

,InvoiceDate,NetFare,ProductType,ItineraryType
0,01-12-2018 11:43,2160,Other Product,Domestic
1,01-12-2018 11:43,0.49,Other Product,Domestic
2,01-12-2018 13:01,6550,Air,Domestic
3,01-12-2018 13:10,6550,Air,Domestic
4,30-11-2018 14:30,5497,Air,Domestic
5,01-12-2018 14:09,4818,Air,Domestic
6,30-11-2018 14:30,1768,Air,Domestic
7,01-12-2018 13:10,7300,Air,Domestic
8,30-11-2018 17:30,7050,Air,Domestic
9,01-12-2018 14:25,7300,Air,Domestic


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.tail(10)

,InvoiceDate,NetFare,ProductType,ItineraryType
278456,10-06-2019 20:05,22427,Air,International
278457,10-06-2019 20:23,1207,Air,Domestic
278458,10-06-2019 20:53,3725,Air,Domestic
278459,10-06-2019 20:32,-4850,Air,Domestic
278460,10-06-2019 20:34,-5492,Air,Domestic
278461,10-06-2019 20:45,0,Air,Domestic
278462,10-06-2019 21:39,9073,Air,Domestic
278463,10-06-2019 22:18,2367,Air,Domestic
278464,10-06-2019 22:30,6572,Air,Domestic
278465,10-06-2019 22:30,5118,Air,Domestic


In [ ]:
data.shape

(278466, 4)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278466 entries, 0 to 278465
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   InvoiceDate    278464 non-null  object
 1   NetFare        217576 non-null  object
 2   ProductType    278464 non-null  object
 3   ItineraryType  245689 non-null  object
dtypes: object(4)
memory usage: 8.5+ MB


# New Section

In [ ]:
data.describe(include='object').transpose()

,count,unique,top,freq
InvoiceDate,278464,97784,30-05-2019 12:26,159
NetFare,217576,31033,0,21931
ProductType,278464,14,Air,151995
ItineraryType,245689,2,Domestic,225231


In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [ ]:
# Just converted the InvoiceDate from object ot DateTime of Pandas

In [ ]:
data['InvoiceDate']

0        2018-01-12 11:43:00
1        2018-01-12 11:43:00
2        2018-01-12 13:01:00
3        2018-01-12 13:10:00
4        2018-11-30 14:30:00
                 ...        
278461   2019-10-06 20:45:00
278462   2019-10-06 21:39:00
278463   2019-10-06 22:18:00
278464   2019-10-06 22:30:00
278465   2019-10-06 22:30:00
Name: InvoiceDate, Length: 278466, dtype: datetime64[ns]

In [ ]:
data['InvoiceDate'].dt.date

0         2018-01-12
1         2018-01-12
2         2018-01-12
3         2018-01-12
4         2018-11-30
             ...    
278461    2019-10-06
278462    2019-10-06
278463    2019-10-06
278464    2019-10-06
278465    2019-10-06
Name: InvoiceDate, Length: 278466, dtype: object

In [ ]:
data['InvoiceDate'].dt.time

0         11:43:00
1         11:43:00
2         13:01:00
3         13:10:00
4         14:30:00
            ...   
278461    20:45:00
278462    21:39:00
278463    22:18:00
278464    22:30:00
278465    22:30:00
Name: InvoiceDate, Length: 278466, dtype: object

In [ ]:
data['InvoiceTime'] = data['InvoiceDate'].dt.time

In [ ]:
data['InvoiceTime'].head(10)

0    11:43:00
1    11:43:00
2    13:01:00
3    13:10:00
4    14:30:00
5    14:09:00
6    14:30:00
7    13:10:00
8    17:30:00
9    14:25:00
Name: InvoiceTime, dtype: object

In [ ]:
data['InvoiceDate'] = data['InvoiceDate'].dt.date

In [ ]:
data['InvoiceDate'].head(10)

0    2018-01-12
1    2018-01-12
2    2018-01-12
3    2018-01-12
4    2018-11-30
5    2018-01-12
6    2018-11-30
7    2018-01-12
8    2018-11-30
9    2018-01-12
Name: InvoiceDate, dtype: object

In [ ]:
type(data['InvoiceDate'][0])

datetime.date

In [ ]:
type(data['InvoiceTime'][0])

datetime.time

In [ ]:
data.head(10)

,InvoiceDate,NetFare,ProductType,ItineraryType,InvoiceTime
0,2018-01-12,2160,Other Product,Domestic,11:43:00
1,2018-01-12,0.49,Other Product,Domestic,11:43:00
2,2018-01-12,6550,Air,Domestic,13:01:00
3,2018-01-12,6550,Air,Domestic,13:10:00
4,2018-11-30,5497,Air,Domestic,14:30:00
5,2018-01-12,4818,Air,Domestic,14:09:00
6,2018-11-30,1768,Air,Domestic,14:30:00
7,2018-01-12,7300,Air,Domestic,13:10:00
8,2018-11-30,7050,Air,Domestic,17:30:00
9,2018-01-12,7300,Air,Domestic,14:25:00


In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [ ]:
(data['InvoiceDate'][data['InvoiceDate'].dt.year <2019]).count()

68786

In [ ]:
(data['InvoiceDate'][data['InvoiceDate'].dt.year <2019]).nunique()

264

In [ ]:
(data['InvoiceDate'][data['InvoiceDate'].dt.year >2018]).count()

209678

In [ ]:
(data['InvoiceDate'][data['InvoiceDate'].dt.year >2018]).nunique()

161

In [ ]:
size = (19,9)

In [ ]:
data[data['InvoiceDate'].dt.year < 2019]['InvoiceDate'].dt.quarter

0         1
1         1
2         1
3         1
4         4
         ..
249615    4
249616    4
249617    4
249618    4
249619    4
Name: InvoiceDate, Length: 68786, dtype: int64

Now converting the NetFare column into Integer

In [ ]:
data['NetFare'].isna().sum()

60890

In [ ]:
if(dateFinder.match('-12-32-1212 :')):
  print("hello world")

In [ ]:
#data['NetFare'] = data['NetFare'].map((lambda x: x*x)(float(x)) for x in range(10))

In [ ]:
dateFinder = re.compile('[0-9]{2}-[0-9]{2}.+')
def FloatConverter(x):
  if type(x)!= float:
    if (dateFinder.match(x)):
      return np.nan
    else:
      return float(x)
  else:
    return np.nan

data['NetFare'] = data['NetFare'].apply(FloatConverter)

TypeError: ignored

In [ ]:
data['NetFare'] = data['NetFare'].astype(float)

ValueError: ignored

In [ ]:
type(data['NetFare'][0])

float

In [ ]:
data['NetFare'].describe().transpose()

count     217576.0
unique     31033.0
top            0.0
freq       21931.0
Name: NetFare, dtype: float64

In [ ]:
data['NetFare'].isna().sum()

60890

In [ ]:
print("Total % Null value in NetFare - ",data['NetFare'].isna().sum()/len(data)*100)

Total % Null value in NetFare -  21.866224242816


In [ ]:
for x in data.columns:
  print("Total % Null value in "+x+" - ",data[x].isna().sum()/len(data)*100)
  print()

Total % Null value in InvoiceDate -  0.0007182205367980292

Total % Null value in NetFare -  21.866224242816

Total % Null value in ProductType -  0.0007182205367980292

Total % Null value in ItineraryType -  11.770557267314501

Total % Null value in InvoiceTime -  0.0007182205367980292



In [ ]:
data['NetFare'].describe()

count     217576.0
unique     31033.0
top            0.0
freq       21931.0
Name: NetFare, dtype: float64

WE can see the Standard Deviation is very far higher than the mean and most of the data are under the 75% percentile and the max value is 49k+

> Indented block


Reason it can be 
Domestic and International fights have different price range 
So we will divide the Domestic and International NetFare value different and then we will analysis the Stats of the NetFare Column

In [ ]:
def DomesticFlight(values):

  domestic_fare = values[values['ItineraryType']=='Domestic']['NetFare']
  print('-'*10,'Domestic Flight Fare','-'*10,'\n')
  print(domestic_fare.describe())

  domestic_air_fare = values[(values['ItineraryType']=='Domestic')&(values['ProductType']=='Air')]['NetFare']
  print('\n\n','-'*10,'Domestic Flight Fare with Air Variable','-'*10,'\n')
  print(domestic_air_fare.describe())

  print(round(domestic_air_fare.describe()[1]))
  
  # returning the DomesticAirFare mean value
  return round(domestic_air_fare.describe()[1])



Dom_mean = DomesticFlight(data)

---------- Domestic Flight Fare ---------- 

count     170387.0
unique     26295.0
top           10.0
freq        5283.0
Name: NetFare, dtype: float64


 ---------- Domestic Flight Fare with Air Variable ---------- 

count     141234
unique     22850
top            0
freq         532
Name: NetFare, dtype: object
22850


In [ ]:
def InternationalFlight(values):

  international_fare = values[values['ItineraryType']=='International']['NetFare']
  print('-'*10,'International Flight Fare','-'*10,'\n')
  print(international_fare.describe())

  international_air_fare = values[(values['ItineraryType']=='International')&(values['ProductType']=='Air')]['NetFare']
  print('\n\n','-'*10,'International Flight Fare with Air Variable','-'*10,'\n')
  print(international_air_fare.describe())

  print(round(international_air_fare.describe()[1]))
  
  # returning the International Air Fare mean value
  return round(international_air_fare.describe()[1])



Inter_mean = InternationalFlight(data)

---------- International Flight Fare ---------- 

count     14413.0
unique     7022.0
top          10.0
freq        923.0
Name: NetFare, dtype: float64


 ---------- International Flight Fare with Air Variable ---------- 

count     10761.0
unique     5994.0
top           0.0
freq         93.0
Name: NetFare, dtype: float64
5994.0


In [ ]:
data[data['NetFare']==0]['NetFare'].count() # Total Number of Zero Value in the NetFare Col

21931

In [ ]:
data[(data['NetFare']==0) & (data['ItineraryType']=='Domestic')]['ItineraryType'].count()
#Total Number of Zero Values for the Corresponding ItineraryType (Domestic)

507

In [ ]:
data[(data['NetFare']==0) & (data['ItineraryType']=='International')]['ItineraryType'].count()
#Total Number of Zero Values for the Corresponding ItineraryType (International)

95

In [ ]:
def ItineraryTypeCheckingClass(allData):
  for x in allData['ItineraryType']:
    if(x==np.nan):
      print(x)
    elif(x!='Domestic') and (x!='International'):
      print(x)
      print(type(x))
      break
    elif(x=='Domestic'):
      print(x)
      print(type(x))
      print()
    elif(x=='International'):
      print(x)
      print(type(x))
      print()

  #we can see that Missing value in this column is Float type and others are string type

#ItineraryTypeCheckingClass(data)

In [ ]:
def ReplacingFloatToNaN(Datas):
  i=1
  for x in range(len(Datas['ItineraryType'])):
    if(type(data.iloc[x,3])== float):
      data.iloc[x,3] = np.nan

#ReplacingFloatToNaN(data)

In [ ]:
#data['ItineraryType'].fillna(method='ffill',inplace=True)

In [ ]:
def ReplaceItineraryTypeNaNValue(values):
  for x in range(len(values['ItineraryType'])):
    if(type(values.iloc[x,3])==float):
      
      if((values.iloc[x,1]) <= 6500):
        values.iloc[x,3] = 'Domestic'

      if((values.iloc[x,1]) > 6500):
        values.iloc[x,3] ='International'
  
  return values
      

#data = ReplaceItineraryTypeNaNValue(data)

We successfully fill the ITINERARY TYPE column 

Now we will fill the NETFARE column

We will use simple aproach to the NaN value 

If we find NaN value in the NETFARE then we check Corresponding ItineraryType column and if the Value of that column is 

DOMESTIC than we will replace with DomesticFlight Mean value 

If, International value then we will replace with InternationalFlight Mean Value

In [ ]:
def FillingNaNValuesOfFlights(values,DomesticMean,InterMean):
  
  for x in range(len(values['NetFare'])):
    if(math.isnan(values.iloc[x,1])):

      if(values.iloc[x,3]=='Domestic'):
        values.iloc[x,1] = DomesticMean
      
      elif(values.iloc[x,3]== 'International'):
        values.iloc[x,1] = Inter_mean
    
  
  return values
      



data = FillingNaNValuesOfFlights(data,Dom_mean,Inter_mean)

In [ ]:
d = DomesticFlight(data). # Filled  NaN with threshold value

In [ ]:
#d = DomesticFlight(data) # Filled with Foward Method

If we compare the Before and After Filling the NaN values of the NetFare Column 
we can see that the mean value is not affected 

so we can go with this 

In [ ]:
for x in data.columns:
  print("Total % Null value in "+x+" - ",data[x].isna().sum()/len(data)*100)
  print()

That 0.00071 Missing % is just 2 Values are missing so we can afford to delete those row from the dataset

In [ ]:
data.dropna(inplace = True)

In [ ]:
len(data)

Year 1 = 2018
Year 2 = 2019

In [ ]:
data['InvoiceDate'].dt.year.value_counts()

In [ ]:
data['Year1']=data[data['InvoiceDate'].dt.year < 2019]['InvoiceDate'].dt.quarter

In [ ]:
data['Year2']=data[data['InvoiceDate'].dt.year > 2018]['InvoiceDate'].dt.quarter

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data.info()

In [ ]:
#data['NetFare'].dropna(inplace = True)

In [ ]:
#data['ProductType'].dropna(inplace = True)

In [ ]:
#data['ItineraryType'].dropna(inplace = True)

In [ ]:
#data['InvoiceTime'].dropna(inplace = True)

In [ ]:
len(data) # Data Befor Droping the NaN Values(278466, 4)

In [ ]:
for x in data.columns:
  print("Total % Null value in "+x+" - ",data[x].isna().sum()/len(data)*100)
  print()

In [ ]:
data.describe().transpose()

In [ ]:
data.describe(include='object').transpose()

In [ ]:
len(data.drop(['Year1','Year2'],1))

In [ ]:
data['InvoiceTime'] = pd.to_datetime(data['InvoiceTime'],format= '%H:%M:%S' ).dt.time

In [ ]:
type(data['InvoiceTime'][0])

In [ ]:
data.describe().transpose()

In [ ]:
data.describe(include='object').transpose()

In [ ]:
data

In [ ]:
#data['InvoiceTime'] = pd.to_timedelta(data['InvoiceTime'].astype(str))

In [ ]:
data.info()

In [ ]:
data.describe(include='datetime')

In [ ]:
data.describe(include='all')

In [ ]:
data['InvoiceDate'].dt.day_name()

In [ ]:
data['InvoiceDay'] = data['InvoiceDate'].dt.day_name()

In [ ]:
data.head(10)

In [ ]:
#data.to_csv('/content/drive/My Drive/ColabDataset/PreProcessed_Concatenate_B2C_B2E.csv',index = False)

In [ ]:
data2 = pd.read_csv('/content/drive/My Drive/ColabDataset/PreProcessed_Concatenate_B2C_B2E.csv')

In [ ]:
data2.head()

In [ ]:
len(data2)